<a href="https://colab.research.google.com/github/owilli38/DSBA-6162/blob/main/DSBA6162_Text_Summarization_ROUGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets rouge_score accelerate sentencepiece

In [ ]:
from datasets import load_dataset

# Load small subsets for training/validation/testing
train_ds = load_dataset("abisee/cnn_dailymail", "3.0.0", split="train[:50]")
val_ds   = load_dataset("abisee/cnn_dailymail", "3.0.0", split="validation[:5]")
test_ds  = load_dataset("abisee/cnn_dailymail", "3.0.0", split="test[:5]")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_input = 512
max_target = 128

def preprocess(batch):
    inputs = ["summarize: " + doc for doc in batch["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input, truncation=True)

    labels = tokenizer(
        batch["highlights"], max_length=max_target, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tok = train_ds.map(preprocess, batched=True)
val_tok = val_ds.map(preprocess, batched=True)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred

    # Decode
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    return {k: v.mid.fmeasure * 100 for k, v in result.items()}


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate

training_args = Seq2SeqTrainingArguments(
    output_dir="./local_t5_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-4,
    predict_with_generate=True,
    logging_steps=10,
    report_to='none'
)

rouge = evaluate.load("rouge")


def compute_metrics(pred):
    preds, labels = pred
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    scores = rouge.compute(predictions=preds, references=labels)
    return {k: v.mid.fmeasure * 100 for k, v in scores.items()}


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-3245788340.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.675700
20,2.625000
30,2.383100
40,1.982500
50,1.921100
60,1.707500
70,1.696000


TrainOutput(global_step=75, training_loss=2.1036844126383465, metrics={'train_runtime': 468.9404, 'train_samples_per_second': 0.32, 'train_steps_per_second': 0.16, 'total_flos': 27661378461696.0, 'train_loss': 2.1036844126383465, 'epoch': 3.0})

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate
import numpy as np

training_args = Seq2SeqTrainingArguments(
    output_dir="./local_t5_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-4,
    predict_with_generate=True,
    logging_steps=10,
    report_to='none'
)

rouge = evaluate.load("rouge")

def compute_metrics(pred):
    preds, labels = pred

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    scores = rouge.compute(predictions=preds, references=labels, use_stemmer=True)

    return {k: v * 100 for k, v in scores.items()}

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Preprocess test examples
test_tok = test_ds.map(preprocess, batched=True)

results = trainer.evaluate(test_tok)
results

/tmp/ipython-input-1335014544.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


{'eval_loss': 1.8658353090286255,
 'eval_model_preparation_time': 0.0108,
 'eval_rouge1': 13.02380952380952,
 'eval_rouge2': 2.307692307692307,
 'eval_rougeL': 12.013227513227513,
 'eval_rougeLsum': 11.568783068783068,
 'eval_runtime': 16.1634,
 'eval_samples_per_second': 0.309,
 'eval_steps_per_second': 0.186}

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate
import numpy as np

training_args = Seq2SeqTrainingArguments(
    output_dir="./local_t5_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-4,
    predict_with_generate=True,
    logging_steps=10,
    report_to='none'
)

rouge = evaluate.load("rouge")

def compute_metrics(pred):
    preds, labels = pred

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    scores = rouge.compute(predictions=preds, references=labels, use_stemmer=True)

    return {k: v * 100 for k, v in scores.items()}


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Preprocess test examples
test_tok = test_ds.map(preprocess, batched=True)

results = trainer.evaluate(test_tok)
results

/tmp/ipython-input-1335014544.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 1.8658353090286255,
 'eval_model_preparation_time': 0.0074,
 'eval_rouge1': 13.02380952380952,
 'eval_rouge2': 2.307692307692307,
 'eval_rougeL': 12.013227513227513,
 'eval_rougeLsum': 11.568783068783068,
 'eval_runtime': 12.9573,
 'eval_samples_per_second': 0.386,
 'eval_steps_per_second': 0.232}

In [ ]:
test_tok = test_ds.map(preprocess, batched=True)

results = trainer.evaluate(test_tok)
results

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 1.8658353090286255,
 'eval_model_preparation_time': 0.0074,
 'eval_rouge1': 13.02380952380952,
 'eval_rouge2': 2.307692307692307,
 'eval_rougeL': 12.013227513227513,
 'eval_rougeLsum': 11.568783068783068,
 'eval_runtime': 16.1831,
 'eval_samples_per_second': 0.309,
 'eval_steps_per_second': 0.185}

In [ ]:
test_article = test_ds[0]["article"]

inputs = tokenizer(
    "summarize: " + test_article,
    return_tensors="pt",
    truncation=True,
    max_length=max_input
).to(model.device)

fine_tuned_summary = tokenizer.decode(
    model.generate(**inputs, max_length=max_target)[0],
    skip_special_tokens=True
)

print("Fine-tuned summary:\n", fine_tuned_summary)

Fine-tuned summary:
 Palestinian Authority officially becomes 123rd member of International Criminal Court . Palestinians signed Rome Statute in January, paving way for possible war crimes investigations . Palestinian Foreign Minister: "I am pleased to announce that Palestine has become the 123rd member of the International Criminal Court"


In [ ]:
from transformers import AutoModelForSeq2SeqLM

orig_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small").to(model.device)

orig_sum = tokenizer.decode(
    orig_model.generate(**inputs, max_length=max_target)[0],
    skip_special_tokens=True
)

print("Original flan-t5-small summary:\n", orig_sum)

Original flan-t5-small summary:
 ICC treaty.
